In [0]:
%run ../../../utils

## Widgets

In [0]:
dbutils.widgets.text("query_input", "")
dbutils.widgets.text("path_landing", "")
dbutils.widgets.text("table_bronze", "")

query_input = dbutils.widgets.get("query_input")
path_landing = dbutils.widgets.get("path_landing")
table_bronze = dbutils.widgets.get("table_bronze")

## Lee query en Azure SQL

In [0]:
df_sql = (
    spark.sql(normalize_query(query_input))
)

## Inserta en landing

In [0]:
def write_landing(
    path: str
) -> DataFrame:
    """
    Lee datos crudos desde la carpeta de Landing en formato Parquet y
    normaliza todas las columnas a tipo string para mantener uniformidad.

    Args:
        path (str): Ruta relativa dentro de la carpeta Landing, por ejemplo:
                    "bronze/sales/CountryRegionCurrency".

    Returns:
        DataFrame: Spark DataFrame con todas las columnas casteadas a string,
                   listo para procesar en la capa Bronze o Silver.
    """
    # Obtiene la ruta base de ADLS Gen2 (abfss://container@storage_account.dfs.core.windows.net)
    path_base = variables_globales()["path_base"]

    # Carga los datos crudos en Parquet desde la ubicación completa
    df = spark.read.format("parquet").load(f"{path_base}/{path}")

    # Para evitar inconsistencias de tipos en downstream, convierte todas las columnas a string
    columns_to_cast = [
        col(c).cast("string").alias(c)  # Mantiene el mismo nombre de columna
        for c in df.columns
    ]

    # Devuelve el DataFrame con el casting aplicado en todas las columnas
    return df.select(*columns_to_cast)
